In [ ]:
#!pip install --upgrade beautifulsoup4

In [ ]:
import requests 
from bs4 import BeautifulSoup
import html.parser
from html.parser import HTMLParser
import sys
import os

import subprocess


Do this

```
rm -rf Class* ; mv ~/Downloads/Class\ List* .
```

In [ ]:
# Open the file.
r = open('Class List.html')

if os.path.exists('./Class List_files'):
    os.rename('Class List_files','Class_List_files')

dirname = 'Class_List_files'

In [ ]:
# Rename the figures

filelist = os.listdir(dirname)

for filename in filelist:
    #print(filename)
    if filename.find('picture') >= 0 and filename.find('.png')<0:
        print(filename)
        os.rename(dirname+"/"+filename, dirname+"/"+filename+'.png')

In [ ]:
#print(output)

In [ ]:
soup = BeautifulSoup(r,"lxml")


In [ ]:
#soup.find_all('appData.courseInfo.courseTitleComposite')
def get_course_name(soup):

    course_name = []
    
    spans = soup.find_all('span')
    for span in spans:
        #contents = span.contents
        #print(contents)
        if 'ng-bind' in list(span.attrs.keys()):
            #print(span.attrs)
            #print(span['ng-bind'])
            if span['ng-bind'] == 'appData.courseInfo.courseTitleComposite':
                #print(span.attrs)
                #print(span['ng-bind'])
                
                for content in span.contents:
                    #print(content.text)
                    course_name.append(content.text)

    return course_name

course_name = get_course_name(soup)

course_name[0] = course_name[0].replace('&','\&')

print(course_name)

#span = spans[0]

#type(span)

#span.contents[0].text

In [ ]:
imgs = soup.find_all('img')

In [ ]:
#img = imgs[0]

In [ ]:
#img

In [ ]:
#img = str(img)

In [ ]:
#img

In [ ]:
#imgs

In [ ]:
lasts,firsts,pics = [], [], []

for img in imgs:
    img = str(img)
    if img.find('ng-src')>=0:
        student_name = img.split('"')[1]
        #print(student_name)
        last,first = student_name.split(',')
        print(first.strip(),last.strip())

        pic = img.split('"')[-2]
        pic = pic.replace(' ','_') + '.png'
        print(pic)

        lasts.append(last.strip())
        firsts.append(first)
        pics.append(pic)

In [ ]:
#pics

In [ ]:
output = ""
output += "\documentclass{article}\n"
output += "\\usepackage{graphicx}\n"
output += "\\usepackage{subfig}\n"
output += "\\usepackage{alphalph}\n"
output += "\\renewcommand\\thesubfigure{\\alphalph{\\value{subfigure}}}\n"

output += "\hoffset=-1.50in\n"
output += "\setlength{\\textwidth}{7.5in}\n"
output += "\setlength{\\textheight}{9in}\n"
output += "\setlength{\\voffset}{0pt}\n"
output += "\setlength{\\topmargin}{0pt}\n"
output += "\setlength{\headheight}{0pt}\n"
output += "\setlength{\headsep}{0pt}\n"
output += "\\begin{document}\n"



icount = 0
rowcount = 0

nrows = 4
ncols = 5
maxsubfigs = nrows*ncols

lbr = '{'
rbr = '}'

closed_figure = False

for first,last,pic in zip(firsts, lasts, pics):
    if first is not None and pic is not None:# and rowcount==5:
        if icount%maxsubfigs==0:
            #print("\\begin{document}")
            output += "\n"
            output += "\\begin{figure}\n"
            output += "\centering\n"
            closed_figure = False

        colcount = icount%ncols
        rowcount = int(icount/ncols)

        print(icount, rowcount, colcount)
        
        last_part = ""
        if colcount < ncols-1:
            last_part = "\\hfill\n"
        else:
            last_part = "\\\\\n"
            
        output += f"\subfloat[{first} {last}]{lbr}\includegraphics[width=0.15\\textwidth]{lbr}{pic}{rbr}{rbr}{last_part}"

        if icount%maxsubfigs==maxsubfigs-1:
            output += f"\caption{lbr}{course_name[0]}{rbr}\n"
            output += "\end{figure}\n"
            closed_figure = True
            icount = -1

        
        icount += 1

if not closed_figure:
    output += f"\caption{lbr}{course_name[0]}{rbr}\n"
    output += "\end{figure}\n"
output += "\end{document}\n"

print(output)

outfile = open('temp.tex','w')
outfile.write(output)
outfile.close()


In [ ]:
# Build th ePF from the LaTeX

subprocess.run(["pdflatex", "temp"]) 

newname = course_name[0].replace(' ','').replace('?','').replace(':','') + '.pdf'
os.rename('temp.pdf', newname)